In [2]:
import networkx as nx

In [13]:
def find_density(G, U):
    H = G.subgraph(U)
    return len(H.edges())/len(H.nodes())


In [9]:
find_density(nx.complete_graph(10), [i for i in range(5)])

2.0

In [10]:
find_density(nx.path_graph(10), [i for i in range(5)])

0.8

In [11]:
find_density(nx.path_graph(10), [i for i in range(0, 10, 2)])

0.0

In [12]:
find_density(nx.complete_graph(10), [i for i in range(10)])

4.5

In [14]:
import math

def maximum_density(G):
    #print(G.nodes())
    max_dens = 0
    for i in range(1, int(math.pow(2, len(G.nodes())))):
        #print("i", i)
        cmb = list(map(int, bin(i)[2:]))
        if len(cmb)<len(G.nodes()):
            cmb = [0 for i in range(len(G.nodes())-len(cmb))] + cmb
        #print(cmb)
        U = []
        i = 0
        for n in G.nodes():
            if cmb[i]==1:
                U.append(n)
            i += 1
        #print(U)
        t = find_density(G, U)
        if t>max_dens:
            max_dens = t
    return max_dens

maximum_density(nx.path_graph(5))


0.8

In [4]:
def restricted_to_edges(E, U, G):
    H = G.subgraph(U)
    E_H = set()
    for e in H.edges():
        u, v = e
        if u>v:
            t = u
            u = v
            v = t
        E_H.add((u, v))
    E_E = set()
    for e in E:
        u, v = e
        if u>v:
            t = u
            u = v
            v = t
        E_E.add((u, v))
    return E_H & E_E

restricted_to_edges([(1, 0), (1, 2)], [0, 1, 2], nx.complete_graph(4))

{(0, 1), (1, 2)}

In [6]:
def cov_G(E, v, G):
    N_v = G[v]
    U = []
    for u in N_v:
        U.append(u)
    return len(restricted_to_edges(E, U, G))
    
cov_G([(1, 0), (1, 2)], 2, nx.path_graph(4))

0

In [25]:
def restricted_neighbor_graph(E, v, G):
    # get nodes
    N_v = G[v]
    U = []
    for u in N_v:
        U.append(u)
    # add edges
    G_out = nx.Graph()
    G_out.add_nodes_from(U)
    H = G.subgraph(U)
    for e in H.edges():
        u, v = e
        G_out.add_edge(u, v)
    # remove edges
    E_st = set()
    for e in E:
        u, v = e
        if u>v:
            t = u
            u = v
            v = t
        E_st.add((u, v))
    G_st = set()
    for e in G_out.edges():
        u, v = e
        if u>v:
            t = u
            u = v
            v = t
        G_st.add((u, v))
    E2 = G_st - E_st
    for e in E2:
        u, v = e
        G_out.remove_edge(u, v)
    return G_out

restricted_neighbor_graph([(1, 0), (1, 2), (2, 3)], 2, nx.path_graph(4))

In [26]:
def max_density_neighbor_graph(E, v, G):
    return maximum_density(restricted_neighbor_graph(E, v, G))

#max_density_neighbor_graph([(1, 0), (1, 2), (2, 3)], 2, nx.path_graph(4))
max_density_neighbor_graph([(1, 0), (1, 2), (2, 3)], 2, nx.complete_graph(4))

0.5